# 🧩 Xenium Spatial Plot Aggregation Notebook

This notebook automates the organization and visualization of spatial transcriptomics results from multiple **Xenium** runs.  
It performs **two main tasks**:

---

## 1️⃣ Generate per-run spatial PDF panels
For each run folder (e.g. `XeniumR1`, `XeniumPR2`, …), the notebook:

- Recursively finds all `spatial_plots.png` images inside subfolders such as `slide1/ROI1/`.
- Creates a **compact, labeled PDF panel** showing all ROIs for that run.
- Each plot is labeled with its **slide** and **ROI** numbers for clarity.
- The resulting file (e.g. `spatial_plots.pdf`) is lightweight and visually consistent.

---

## 2️⃣ Combine multiple runs into a single summary PDF
After generating the per-run PDFs, the notebook merges them into one consolidated report:

- Sequentially appends all run PDFs (e.g. `XeniumR1` → `XeniumR6` → `XeniumPR3`).
- Automatically **rescales pages to a uniform width** for a clean, standardized layout.
- Produces a single combined file (e.g. `spatial_plots_R_PR.pdf`) summarizing all runs.

---

## 🧠 Typical workflow
1. **Set the root folder** containing all run directories, e.g.  
   `/project/.../xenium_data/`
2. **Run the panel generation cell** to create one PDF per Xenium folder.
3. **Run the combination cell** to merge all run PDFs into one summary.

---

This notebook streamlines visualization across multiple Xenium experiments — enabling quick review, archiving, or sharing of large sets of spatial plots in a single, standardized PDF.


In [1]:
#!pip install --quiet pypdf reportlab
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import math
from hest.vis.spatial_plot import *

In [2]:
# ---- Base folder ----
base_root = Path("/project/simmons_hts/kxu/hest/xenium_data")

# Datasets to inspect
#datasets = ['XeniumPR1_segger'] + [f"XeniumPR{i}" for i in range(2, 6)]
datasets = [f"XeniumPR{i}_cell" for i in range(1, 4)]

# ---- Parameters ----
cols = 4         # number of columns in grid
max_show = None    # limit per dataset (set None for all)
thumb_size = (800, 800)  # resize to speed up plotting

for ds in datasets:
    ds_dir = base_root / ds
    if not ds_dir.exists():
        print(f"⚠️ {ds_dir} not found, skipping.")
        continue

    found = sorted(ds_dir.rglob("spatial_plots.png"))
    if not found:
        print(f"⚠️ No spatial_plots.png found in {ds_dir}")
        continue

    if max_show:
        found = found[:max_show]

    print(f"🧩 {ds}: visualising {len(found)} plots")

    # --- build grid ---
    rows = math.ceil(len(found) / cols)
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
    axes = axes.flatten()

    for i, (ax, path) in enumerate(zip(axes, found)):
        try:
            img = Image.open(path)
            img.thumbnail(thumb_size)
            ax.imshow(img)
            rel = path.relative_to(ds_dir)
            ax.set_title(str(rel.parent), fontsize=8)
            ax.axis("off")
        except Exception as e:
            ax.axis("off")
            ax.text(0.5, 0.5, f"Error\n{e}", ha="center", va="center", fontsize=8)

    # hide unused subplots
    for j in range(len(found), len(axes)):
        axes[j].axis("off")

    plt.tight_layout()

    # --- save to PDF under dataset folder ---
    out_pdf = ds_dir / "spatial_plots.pdf"
    plt.savefig(out_pdf, bbox_inches="tight")
    plt.close(fig)
    print(f"✔ Saved grid to {out_pdf}\n")

print("✅ Done generating all spatial_plots.pdf files.")


🧩 XeniumPR1_cell: visualising 15 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/xenium_data/XeniumPR1_cell/spatial_plots.pdf

🧩 XeniumPR2_cell: visualising 8 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/xenium_data/XeniumPR2_cell/spatial_plots.pdf

🧩 XeniumPR3_cell: visualising 8 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/xenium_data/XeniumPR3_cell/spatial_plots.pdf

✅ Done generating all spatial_plots.pdf files.


In [48]:
# Imports and helper functions
from pathlib import Path
import tempfile
from pypdf import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import portrait
import fnmatch
import os
import io

def make_title_page_bytes(title: str, width_pt: float, height_pt: float, font_size: int = 36) -> bytes:
    """Create a one-page PDF (bytes) with `title` centered using ReportLab."""
    buf_path = Path(tempfile.gettempdir()) / f"title_{abs(hash(title)) & 0xffffffff}.pdf"
    c = canvas.Canvas(str(buf_path), pagesize=(width_pt, height_pt))
    x = width_pt / 2.0
    y = height_pt / 2.0
    c.setFont("Helvetica-Bold", font_size)
    # wrap/truncate long titles
    max_chars = max(10, int(width_pt // (font_size * 0.55)))
    if len(title) > max_chars:
        title = title[:max_chars-3] + "..."
    c.drawCentredString(x, y, title)
    c.showPage()
    c.save()
    return buf_path.read_bytes()


def resolve_folders(root: Path, names: list) -> list:
    root = Path(root).expanduser().resolve()
    if not root.exists() or not root.is_dir():
        raise FileNotFoundError(f"Root folder does not exist or is not a directory: {root}")
    elif names:
        folders = []
        for n in names:
            p = root / n
            if p.exists() and p.is_dir():
                folders.append(p)
            else:
                print(f"Warning: folder not found or not a directory, skipping: {p}")
    else:
        raise ValueError("You must specify folders, or use ALL or PATTERN.")
    return folders

TARGET_WIDTH_PT = 400.0  # adjust desired uniform width (pts)

def combine_folder_pdf(folder: Path, out_writer: PdfWriter, title_font_size: int = 36) -> bool:
    pdf_path = folder / "spatial_plots.pdf"
    if not pdf_path.exists():
        return False

    reader = PdfReader(str(pdf_path))
    if len(reader.pages) == 0:
        return False

    # title page (same width)
    title_h = 120.0
    title_bytes = make_title_page_bytes(folder.name, TARGET_WIDTH_PT, title_h, font_size=title_font_size)
    title_reader = PdfReader(io.BytesIO(title_bytes))
    out_writer.add_page(title_reader.pages[0])

    for page in reader.pages:
        try:
            ow = float(page.mediabox.width)
            oh = float(page.mediabox.height)
        except Exception:
            ow, oh = TARGET_WIDTH_PT, TARGET_WIDTH_PT * 0.75

        scale = TARGET_WIDTH_PT / ow

        # try to scale the page in-place
        try:
            page.scale_by(scale)
        except Exception:
            # fallback: leave unscaled (rare)
            pass

        new_w = float(page.mediabox.width)
        new_h = float(page.mediabox.height)

        # create a blank page of the scaled page size (width will equal TARGET_WIDTH_PT)
        blank = out_writer.add_blank_page(width=new_w, height=new_h)

        # merge the scaled page onto the blank at (0,0)
        try:
            blank.merge_translated_page(page, 0, 0)
        except Exception:
            try:
                blank.mergeScaledTranslatedPage(page, 1.0, 0, 0)
            except Exception:
                # last resort: append page directly
                out_writer.add_page(page)

    return True



In [3]:
# Set the root folder that contains the Xenium subfolders (change this path):
ROOT_DIR = "/project/simmons_hts/kxu/hest/xenium_data"    

# list specific folder names to include (relative names)
FOLDERS = ["XeniumR1",'XeniumR2',"XeniumR3",
           "XeniumR5","XeniumR6", 'XeniumPR1',
           "XeniumPR2","XeniumPR3",'XeniumPR4','XeniumPR5',
           'XeniumPR1_cell',"XeniumPR2_cell","XeniumPR3_cell"]  


# Output file
output_filename = "/project/simmons_hts/kxu/hest/xenium_data/spatial_plots/spatial_plots_R_PR.pdf"  # will be created in current working directory, or give full path

TITLE_FONT_SIZE = 36

# ----------------------------
from pypdf import PdfWriter
from pathlib import Path

root = Path(ROOT_DIR)
folders = resolve_folders(root, FOLDERS)
print(f"Found {len(folders)} folders to check.")

writer = PdfWriter()
appended_any = False
missing = []

for f in folders:
    ok = combine_folder_pdf(f, writer, title_font_size=TITLE_FONT_SIZE)
    if ok:
        appended_any = True
        print(f"Appended: {f}/spatial_plots.pdf (title page: '{f.name}')")
    else:
        missing.append(f)
        print(f"Missing or empty: {f}/spatial_plots.pdf — skipped")

if not appended_any:
    print("No PDFs were appended. Exiting without writing output.")
else:
    out_path = Path(output_filename)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with open(out_path, "wb") as of:
        writer.write(of)
    print(f"Combined PDF written to: {out_path.resolve()}")

if missing:
    print("\nFolders skipped (no spatial_plots.pdf found):")
    for m in missing:
        print("  -", m)


Found 13 folders to check.
Appended: /ceph/project/simmons_hts/kxu/hest/xenium_data/XeniumR1/spatial_plots.pdf (title page: 'XeniumR1')
Appended: /ceph/project/simmons_hts/kxu/hest/xenium_data/XeniumR2/spatial_plots.pdf (title page: 'XeniumR2')
Appended: /ceph/project/simmons_hts/kxu/hest/xenium_data/XeniumR3/spatial_plots.pdf (title page: 'XeniumR3')
Appended: /ceph/project/simmons_hts/kxu/hest/xenium_data/XeniumR5/spatial_plots.pdf (title page: 'XeniumR5')
Appended: /ceph/project/simmons_hts/kxu/hest/xenium_data/XeniumR6/spatial_plots.pdf (title page: 'XeniumR6')
Appended: /ceph/project/simmons_hts/kxu/hest/xenium_data/XeniumPR1/spatial_plots.pdf (title page: 'XeniumPR1')
Appended: /ceph/project/simmons_hts/kxu/hest/xenium_data/XeniumPR2/spatial_plots.pdf (title page: 'XeniumPR2')
Appended: /ceph/project/simmons_hts/kxu/hest/xenium_data/XeniumPR3/spatial_plots.pdf (title page: 'XeniumPR3')
Appended: /ceph/project/simmons_hts/kxu/hest/xenium_data/XeniumPR4/spatial_plots.pdf (title pag

# Generate patch visualisation

In [5]:
# ---- Base folder ----
base_root = Path("/project/simmons_hts/kxu/hest/xenium_data")

# Datasets to inspect
datasets = [f"XeniumPR{i}" for i in range(1, 6)] + [f"XeniumPR{i}_cell" for i in range(1, 4)] + [f"XeniumR{i}" for i in range(1, 7)] 

# ---- Parameters ----
cols = 4         # number of columns in grid
max_show = None    # limit per dataset (set None for all)
thumb_size = (800, 800)  # resize to speed up plotting

for ds in datasets:
    ds_dir = base_root / ds
    if not ds_dir.exists():
        print(f"⚠️ {ds_dir} not found, skipping.")
        continue

    # find any filename that contains "patch_vis" (covers ROI9_patch_vis.png etc.)
    found = sorted(ds_dir.rglob("*patch_vis*.png"))
    if not found:
        print(f"⚠️ No *patch_vis*.png found in {ds_dir}")
        continue

    if max_show:
        found = found[:max_show]

    print(f"🧩 {ds}: visualising {len(found)} plots")

    # --- build grid ---
    rows = math.ceil(len(found) / cols)
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
    axes = axes.flatten()

    for i, (ax, path) in enumerate(zip(axes, found)):
        try:
            img = Image.open(path)
            img.thumbnail(thumb_size)
            ax.imshow(img)
            rel = path.relative_to(ds_dir)
            ax.set_title(str(rel.parent), fontsize=8)
            ax.axis("off")
        except Exception as e:
            ax.axis("off")
            ax.text(0.5, 0.5, f"Error\n{e}", ha="center", va="center", fontsize=8)

    # hide unused subplots
    for j in range(len(found), len(axes)):
        axes[j].axis("off")

    plt.tight_layout()

    # --- save to PDF under dataset folder ---
    out_pdf = ds_dir / "patch_vis.pdf"
    plt.savefig(out_pdf, bbox_inches="tight")
    plt.close(fig)
    print(f"✔ Saved grid to {out_pdf}\n")

print("✅ Done generating all patch_vis.pdf files.")

🧩 XeniumPR1: visualising 15 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/xenium_data/XeniumPR1/patch_vis.pdf

🧩 XeniumPR2: visualising 8 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/xenium_data/XeniumPR2/patch_vis.pdf

🧩 XeniumPR3: visualising 8 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/xenium_data/XeniumPR3/patch_vis.pdf

🧩 XeniumPR4: visualising 20 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/xenium_data/XeniumPR4/patch_vis.pdf

🧩 XeniumPR5: visualising 21 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/xenium_data/XeniumPR5/patch_vis.pdf

🧩 XeniumPR1_cell: visualising 15 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/xenium_data/XeniumPR1_cell/patch_vis.pdf

🧩 XeniumPR2_cell: visualising 8 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/xenium_data/XeniumPR2_cell/patch_vis.pdf

🧩 XeniumPR3_cell: visualising 8 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/xenium_data/XeniumPR3_cell/patch_vis.pdf

🧩 XeniumR1: visualising 8 plots
✔ Save

In [6]:
def combine_folder_pdf(folder: Path, out_writer: PdfWriter, title_font_size: int = 36) -> bool:
    pdf_path = folder / "patch_vis.pdf"
    if not pdf_path.exists():
        return False

    reader = PdfReader(str(pdf_path))
    if len(reader.pages) == 0:
        return False

    # title page (same width)
    title_h = 120.0
    title_bytes = make_title_page_bytes(folder.name, TARGET_WIDTH_PT, title_h, font_size=title_font_size)
    title_reader = PdfReader(io.BytesIO(title_bytes))
    out_writer.add_page(title_reader.pages[0])

    for page in reader.pages:
        try:
            ow = float(page.mediabox.width)
            oh = float(page.mediabox.height)
        except Exception:
            ow, oh = TARGET_WIDTH_PT, TARGET_WIDTH_PT * 0.75

        scale = TARGET_WIDTH_PT / ow

        # try to scale the page in-place
        try:
            page.scale_by(scale)
        except Exception:
            # fallback: leave unscaled (rare)
            pass

        new_w = float(page.mediabox.width)
        new_h = float(page.mediabox.height)

        # create a blank page of the scaled page size (width will equal TARGET_WIDTH_PT)
        blank = out_writer.add_blank_page(width=new_w, height=new_h)

        # merge the scaled page onto the blank at (0,0)
        try:
            blank.merge_translated_page(page, 0, 0)
        except Exception:
            try:
                blank.mergeScaledTranslatedPage(page, 1.0, 0, 0)
            except Exception:
                # last resort: append page directly
                out_writer.add_page(page)

    return True

# Set the root folder that contains the Xenium subfolders (change this path):
ROOT_DIR = "/project/simmons_hts/kxu/hest/xenium_data"    

# list specific folder names to include (relative names)
FOLDERS = ["XeniumR1",'XeniumR2',"XeniumR3",
           "XeniumR5","XeniumR6", 'XeniumPR1',
           "XeniumPR2","XeniumPR3",'XeniumPR4','XeniumPR5',
          'XeniumPR1_cell',"XeniumPR2_cell","XeniumPR3_cell"]  


# Output file
output_filename = "/project/simmons_hts/kxu/hest/xenium_data/spatial_plots/patch_vis_R_PR.pdf"  # will be created in current working directory, or give full path

TITLE_FONT_SIZE = 36

# ----------------------------
from pypdf import PdfWriter
from pathlib import Path

root = Path(ROOT_DIR)
folders = resolve_folders(root, FOLDERS)
print(f"Found {len(folders)} folders to check.")

writer = PdfWriter()
appended_any = False
missing = []

for f in folders:
    ok = combine_folder_pdf(f, writer, title_font_size=TITLE_FONT_SIZE)
    if ok:
        appended_any = True
        print(f"Appended: {f}/spatial_plots.pdf (title page: '{f.name}')")
    else:
        missing.append(f)
        print(f"Missing or empty: {f}/spatial_plots.pdf — skipped")

if not appended_any:
    print("No PDFs were appended. Exiting without writing output.")
else:
    out_path = Path(output_filename)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with open(out_path, "wb") as of:
        writer.write(of)
    print(f"Combined PDF written to: {out_path.resolve()}")

if missing:
    print("\nFolders skipped (no spatial_plots.pdf found):")
    for m in missing:
        print("  -", m)

Found 13 folders to check.
Appended: /ceph/project/simmons_hts/kxu/hest/xenium_data/XeniumR1/spatial_plots.pdf (title page: 'XeniumR1')
Appended: /ceph/project/simmons_hts/kxu/hest/xenium_data/XeniumR2/spatial_plots.pdf (title page: 'XeniumR2')
Appended: /ceph/project/simmons_hts/kxu/hest/xenium_data/XeniumR3/spatial_plots.pdf (title page: 'XeniumR3')
Appended: /ceph/project/simmons_hts/kxu/hest/xenium_data/XeniumR5/spatial_plots.pdf (title page: 'XeniumR5')
Appended: /ceph/project/simmons_hts/kxu/hest/xenium_data/XeniumR6/spatial_plots.pdf (title page: 'XeniumR6')
Appended: /ceph/project/simmons_hts/kxu/hest/xenium_data/XeniumPR1/spatial_plots.pdf (title page: 'XeniumPR1')
Appended: /ceph/project/simmons_hts/kxu/hest/xenium_data/XeniumPR2/spatial_plots.pdf (title page: 'XeniumPR2')
Appended: /ceph/project/simmons_hts/kxu/hest/xenium_data/XeniumPR3/spatial_plots.pdf (title page: 'XeniumPR3')
Appended: /ceph/project/simmons_hts/kxu/hest/xenium_data/XeniumPR4/spatial_plots.pdf (title pag

# generate Visium plots

In [2]:
base_root = Path("/project/simmons_hts/kxu/hest/visium_data")
datasets = [f"VisiumR{i}" for i in range(1, 7)]

make_spatial_plot_per_run(base_root, datasets)

🧩 VisiumR1: visualising 4 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/visium_data/VisiumR1/spatial_plots.pdf

🧩 VisiumR2: visualising 8 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/visium_data/VisiumR2/spatial_plots.pdf

🧩 VisiumR3: visualising 4 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/visium_data/VisiumR3/spatial_plots.pdf

🧩 VisiumR4: visualising 4 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/visium_data/VisiumR4/spatial_plots.pdf

🧩 VisiumR5: visualising 7 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/visium_data/VisiumR5/spatial_plots.pdf

🧩 VisiumR6: visualising 8 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/visium_data/VisiumR6/spatial_plots.pdf

✅ Done generating all spatial_plots.pdf files.


In [3]:
base_root = Path("/project/simmons_hts/kxu/hest/visium_data")
datasets = [f"VisiumR{i}" for i in range(1, 7)]

make_patch_vis_per_run(base_root, datasets)

🧩 VisiumR1: visualising 4 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/visium_data/VisiumR1/patch_vis.pdf

🧩 VisiumR2: visualising 8 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/visium_data/VisiumR2/patch_vis.pdf

🧩 VisiumR3: visualising 4 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/visium_data/VisiumR3/patch_vis.pdf

🧩 VisiumR4: visualising 4 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/visium_data/VisiumR4/patch_vis.pdf

🧩 VisiumR5: visualising 7 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/visium_data/VisiumR5/patch_vis.pdf

🧩 VisiumR6: visualising 7 plots
✔ Saved grid to /project/simmons_hts/kxu/hest/visium_data/VisiumR6/patch_vis.pdf

✅ Done generating all patch_vis.pdf files.


In [2]:
# combine spatial plots
combine_pdfs(root_dir = "/project/simmons_hts/kxu/hest/visium_data",
             folder_names = ["VisiumR1","VisiumR2","VisiumR3",
                  "VisiumR4","VisiumR5","VisiumR6"],
             output_filename = "/project/simmons_hts/kxu/hest/visium_data/spatial_plots/spatial_plots.pdf", 
             pdf_name="spatial_plots.pdf")

# combine patch visualizations
#combine_pdfs(root_dir, folders, out_file, pdf_name="patch_vis.pdf")

Found 6 folders to check.
Appended: /ceph/project/simmons_hts/kxu/hest/visium_data/VisiumR1/spatial_plots.pdf (title page: 'VisiumR1')
Appended: /ceph/project/simmons_hts/kxu/hest/visium_data/VisiumR2/spatial_plots.pdf (title page: 'VisiumR2')
Appended: /ceph/project/simmons_hts/kxu/hest/visium_data/VisiumR3/spatial_plots.pdf (title page: 'VisiumR3')
Appended: /ceph/project/simmons_hts/kxu/hest/visium_data/VisiumR4/spatial_plots.pdf (title page: 'VisiumR4')
Appended: /ceph/project/simmons_hts/kxu/hest/visium_data/VisiumR5/spatial_plots.pdf (title page: 'VisiumR5')
Appended: /ceph/project/simmons_hts/kxu/hest/visium_data/VisiumR6/spatial_plots.pdf (title page: 'VisiumR6')
Combined PDF written to: /ceph/project/simmons_hts/kxu/hest/visium_data/spatial_plots/spatial_plots.pdf
